In [ ]:
import numpy as np
from dataframe import Data
from SNANA_FITS_to_pd import read_fits
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib 
from io_utils import *
from random import random
%load_ext memory_profiler
from filter import transient_filter_load_saved
from filter import transient_filter

In [ ]:
dataset_val = 1

In [ ]:
if dataset_val == 0:
    dataset =  "PLAsTiCC"
    data_ob = load_PLAsTiCC_data()
    object_ids = data_ob.get_all_object_ids()
    kilonova_ids=data_ob.get_ids_of_event_type(64)
    color_band_dict = {0:'C4',1:'C2', 2:'C3', 3:'C1', 4:'k', 5:'C5'}
    bands = [0,1,2,3,4,5]
    transient_filter_load_saved(data_ob=data_ob)

In [ ]:
#print(data_ob.df_metadata['filter_result'])

In [ ]:
if dataset_val == 1:    
    dataset = "ZTF"
    data_ob = load_ztf_mixed()
    object_ids = data_ob.get_all_object_ids()
    kilonova_ids = object_ids
    bands = [b'g ', b'r ']
    color_band_dict = {b'g ':'C2', b'r ':'C3'}
    #transient_filter(data_ob=data_ob)

In [ ]:
from LightCurve import LightCurve

In [ ]:
def get_band_max_point(band_df):
    loc = np.argmax(band_df[data_ob.flux_col_name])
    date = band_df[data_ob.time_col_name][loc]
    max_flux = band_df[data_ob.flux_col_name][loc]
    return date, max_flux

In [ ]:
def get_PCs(num_components, all_bands= False, band_choice='z'):
    
    if all_bands == True:
        PC_dict = np.load("principal_components/PC_all_bands_diff_mid_pt_dict.npy")
        PC_dict = PC_dict.item()
        PC_out = {}
        #num_components = int(num_components)
        #print(PC_dict['u'])
        PC_out[0] = PC_dict['u'][0:num_components]
        PC_out[1] = PC_dict['r'][0:num_components]
        PC_out[2] = PC_dict['i'][0:num_components]
        PC_out[3] = PC_dict['g'][0:num_components]
        PC_out[4] = PC_dict['z'][0:num_components]
        PC_out[5] = PC_dict['Y'][0:num_components]
        
    else:
        PC_out={}
        PC_dict = np.load("principal_components/PC_all_bands_diff_mid_pt_dict.npy")
        PC_dict = PC_dict.item()
        for band in data_ob.band_map.keys():
            PC_out[band] = PC_dict[band_choice][0:num_components]
            
    return PC_out

In [ ]:
def optimize_coeff(band_df, mid_point_date, current_date, PCs, no_of_predicted_days=51, time_step = 2):
    
    if(len(band_df)>0): 

        start_date=mid_point_date-(no_of_predicted_days-1)*time_step/2 
        end_date= mid_point_date+(no_of_predicted_days-1)*time_step/2 
        start_index = band_df[data_ob.time_col_name] >= start_date
        end_index = band_df[data_ob.time_col_name] <= end_date 
        
        past_index = band_df[data_ob.time_col_name] <= current_date 
        fit_df = band_df[start_index&end_index&past_index]
        
        if len(fit_df)>0:
            
            binned_dates = get_binned_time(fit_df)
            b2 = (binned_dates-mid_point_date+no_of_predicted_days-1)/2
            b2 = b2.astype(int)
            light_curve_seg = np.zeros((no_of_predicted_days))
            light_curve_seg[b2[:]] = fit_df[data_ob.flux_col_name]
            #initial_guess = np.amax(fit_df[data_ob.flux_col_name])*np.array([.93,.03 ,.025])
            initial_guess = [.93,.03 ,.025]
            result = minimize(calc_loss, initial_guess, args=(PCs, light_curve_seg))
            
            return result.x
        
        
            predicted_lt_curve = calc_prediction(result.x,PCs)
            x_data = np.arange(start_date,end_date+2,2)
            
            return x_data, predicted_lt_curve, result.x
        
    return []
            

In [ ]:
def calc_prediction(coeff, PCs, bias = None):
    predicted_lc = np.zeros_like(PCs.shape[1])
    for a,b in zip(PCs,coeff): predicted_lc = np.add(predicted_lc, b*a)
    if bias is not None:
        predicted_lc = predicted_lc+bias
    return predicted_lc

In [ ]:
def calc_loss(coeff, PCs, light_curve_seg, bias = None):
    index = light_curve_seg!=0
    y_pred = calc_prediction(coeff,PCs, bias = bias)
    diff = light_curve_seg - y_pred
    neg_index = y_pred<0
    diff = diff[(index)|(neg_index)]
    
    #diff = diff[index]
    
    #error = np.sum(np.square(diff,diff))  np.sum(np.square(regularization_vals-coeff/np.sum(coeff)))*1000
    error = np.sum(np.square(diff,diff))
    return error

In [ ]:
def add_to_coeff_arr(coeff_arr, coeff_dict):
    all_coeff = np.array([])
    for key, value in coeff_dict.items():
        if all_coeff.size==0:
            all_coeff = np.asarray(value)
        else:
            all_coeff= np.concatenate((all_coeff, value), axis = 0)
    #print(all_coeff)
        
    if all_coeff.size!=0:
        coeff_arr.append(all_coeff)
        event_type.append(data_ob.get_object_type_number(object_id))
    return coeff_arr

In [ ]:
#print(len(object_ids))

def get_binned_time(df):
    return df[data_ob.time_col_name]-df[data_ob.time_col_name]%2

In [ ]:
from scipy.optimize import minimize

In [ ]:
def get_time_segment(event_df, start_date, end_date, current_date=None):

    start_index = event_df[data_ob.time_col_name] >= start_date
    end_index = event_df[data_ob.time_col_name] <= end_date 
    if current_date is None:
        return event_df[start_index&end_index]
    else:
        past_index = event_df[data_ob.time_col_name] <= current_date 
        return event_df[start_index&end_index&past_index]

In [ ]:
def predict_lc_coeff(event_df, PC_dict, current_date= None, no_of_predicted_days = 51, time_step=2, bands=None, separate_band_mid= True, min_flux_threshold=30):
    coeff_all_band = {}
    
    if bands is None:
        bands = data_ob.band_map.keys()
        
    #print(bands)
    mid_point_dict = get_mid_pt(event_df, bands, current_date, color_band_dict, separate_band_mid, min_flux_threshold=min_flux_threshold)
    #print(mid_point_dict)
    
    #print(bands)
    #print(mid_point_date)
    if mid_point_dict is not None:
        
        for band in bands:
            mid_point_date = mid_point_dict[band]
            if mid_point_date is None:
                coeff_all_band[band] = [0, 0, 0]
                continue
                
            prediction_start_date=mid_point_date-(no_of_predicted_days-1)*time_step/2 
            prediction_end_date= mid_point_date+(no_of_predicted_days-1)*time_step/2 

            event_df = get_time_segment(event_df, prediction_start_date, prediction_end_date, current_date)
        
            band_index = event_df[data_ob.band_col_name] == band
            band_df = event_df[band_index]
            #print(band_df)
            PCs = PC_dict[band]
            if len(band_df)>0:
                
                binned_dates = get_binned_time(band_df)
                b2 = (binned_dates-mid_point_date+no_of_predicted_days-1)/2
                b2 = b2.astype(int)
                light_curve_seg = np.zeros((no_of_predicted_days))
                light_curve_seg[b2[:]] = band_df[data_ob.flux_col_name]
                initial_guess = np.amax(band_df[data_ob.flux_col_name])*np.array([.93,.03 ,.025])
                result = minimize(calc_loss, initial_guess, args=(PCs, light_curve_seg))
                coeff_all_band[band] = list(result.x)
                
            else:
                coeff_all_band[band] = [0, 0, 0]
                
    else:
        for band in bands:
            coeff_all_band[band] = [0, 0, 0]
                
    return coeff_all_band
            
    
    

In [ ]:
import time

In [ ]:
print(len(object_ids))

In [ ]:
def get_mid_pt(event_df, bands, current_date=None, color_band_dict=None, separate_band_mid = True, min_flux_threshold =30):
    
    mid_point_dict = {}
    
    if current_date is not None:
        date_difference = event_df[data_ob.time_col_name] - current_date
        past_index = (date_difference>=-50) & (date_difference<=0)
        event_df = event_df[past_index]
        #print(event_df)
        if not separate_band_mid:
            band_mid_points = []
            for i,band in enumerate(bands):
                #print(band)
                band_index = event_df[data_ob.band_col_name] == band
                band_df = event_df[band_index]
                #print(band_df)
                if(len(band_df)>0):
                    max_index = np.argmax(band_df[data_ob.flux_col_name])
                    band_mid_points.append(band_df[data_ob.time_col_name][max_index])
            if len(band_mid_points)>0:
                for band in bands:
                    mid_point_dict[band] = np.median(np.array(band_mid_points))
            else:
                return None
        else:
            for band in bands:
                #print(band)
                band_index = event_df[data_ob.band_col_name] == band
                band_df = event_df[band_index]
                #print(band_df)
                if(len(band_df)>0):
                    max_index = np.argmax(band_df[data_ob.flux_col_name])
                    if band_df[data_ob.flux_col_name][max_index]>min_flux_threshold:
                        mid_point_dict[band] = band_df[data_ob.time_col_name][max_index]
                    else:
                        mid_point_dict[band] = None
                else:
                    mid_point_dict[band] = None
         
        
    else:
        lc = LightCurve(event_df, time_col_name=data_ob.time_col_name, brightness_col_name=data_ob.flux_col_name, brightness_err_col_name=data_ob.flux_err_col_name,band_col_name=data_ob.band_col_name, band_map=data_ob.band_map)
        priority_regions = lc.find_region_priority()
        priority_region1 = priority_regions[0]
        median = np.median(np.asarray(priority_region1))
        event_df = get_time_segment(event_df, median-50, median+50)

        if separate_band_mid:
            
            for band in bands:
                #print(band)
                band_df = lc.extract_band_data(band,event_df)
                #print(band_df)
                if(len(band_df)>0):
                    max_index = np.argmax(band_df[data_ob.flux_col_name])
                    #print(min_flux_threshold)
                    if band_df[data_ob.flux_col_name][max_index]>min_flux_threshold:
                        mid_point_dict[band] = band_df[data_ob.time_col_name][max_index]
                    else: mid_point_dict[band] = None
                else: mid_point_dict[band] = None
        else:
            for band in bands:
                band_df = lc.extract_band_data(band,event_df)
                if len(band_df)>0:
                    #print(min_flux_threshold)
                    if np.amax(band_df[data_ob.flux_col_name]>min_flux_threshold):
                        mid_point_dict[band] = median
                    else: mid_point_dict[band] = None
                else: mid_point_dict[band] = None
    
    #print(mid_point_dict)
    
    return mid_point_dict

In [ ]:
def plot_predicted_bands(all_band_coeff_dict, PC_dict, current_date=None, bands=None, mark_maximum = False, x_lims=None, separate_band_mid=True, min_flux_threshold =30):
    if bands is None: 
        bands = data_ob.band_map.keys()
    mid_point_dict = get_mid_pt(event_df, bands, current_date, color_band_dict, separate_band_mid, min_flux_threshold=min_flux_threshold)
    #print(separate_band_mid)
    lc = LightCurve(event_df, time_col_name=data_ob.time_col_name, brightness_col_name=data_ob.flux_col_name, brightness_err_col_name=data_ob.flux_err_col_name,band_col_name=data_ob.band_col_name, band_map=data_ob.band_map)
    fig = lc.plot_light_curve(color_band_dict=color_band_dict, alpha=0.3, mark_maximum = False, mark_label= False, plot_points = True)
    
            
                    
    if mid_point_dict is not None:
        for band, coeff in all_band_coeff_dict.items():
            mid_point_date = mid_point_dict[band]
            #print(mid_point_date)
            if mid_point_date is not None:
                
                if current_date is None:
                    end_date = mid_point_date +50
                else: 
                    end_date = current_date
                    
                if mark_maximum:
                    fig = lc.plot_light_curve(color_band_dict, fig = fig, start_date= mid_point_date -50, end_date=end_date, band = band, alpha=1, mark_maximum = True, plot_points = True)
                else:
                    fig = lc.plot_light_curve(color_band_dict, fig = fig, start_date= mid_point_date -50, end_date=end_date, band = band, alpha=1, mark_maximum = False, plot_points = True)   

                if len(coeff)!=0:
                    predicted_lc= calc_prediction(coeff,PC_dict[band])
                    #plt.plot(x_data, predicted_lc, color = color_band_dict[band])
                    time_data= np.arange(0,102,2) + mid_point_date - 50
                else: 
                    predicted_lc=[]
                    time_data=[]

                plt.plot(time_data, predicted_lc, color = color_band_dict[band])

        if x_lims is not None: 
            plt.xlim(x_lims)
        
        if not separate_band_mid:
            _, _, ymin, ymax = plt.axis()    
            plt.plot([mid_point_date,mid_point_date],[ymin/2,ymax/2],color = "slateblue", ls="dashed", label="median of max dates")
    
    xmin, xmax, ymin, ymax = plt.axis() 
    #plt.plot([xmin,xmax],[min_flux_threshold,min_flux_threshold],color= "red", ls="dashed", label="min flux threshold ="+str(min_flux_threshold))
    if current_date is not None:
        plt.plot([current_date,current_date],[ymin/2,ymax/2],color= "darkorange", ls="dashed", label="current date")
        
    ax=plt.gca()
    plt.text(.01,.94,data_ob.object_id_col_name+": "+str(object_id),fontsize=15, transform=ax.transAxes)
    if dataset_val == 0:
        #print(data_ob.get_object_type_for_PLAsTiCC(object_id))
        plt.text(.01,.88,"Type: "+data_ob.get_object_type_for_PLAsTiCC(object_id),fontsize=15, transform=ax.transAxes)
    
    plt.xlabel("mjd", fontsize=20)
    plt.ylabel("flux", fontsize=20)
    
    plt.legend(loc="upper right")
    

# plots assuming we have all data

In [ ]:
PC_dict = get_PCs(3,all_bands=False)
start = time.time()
coeff_arr = []
event_type = []
#object_ids = data_ob.get_ids_of_event_type(6)
fig = plt.figure()
#ax = fig.gca(projection='3d')
coeff_array = []
coeff_dict_list=[]
separate_band_mid = True
mark_maximum = False
use_filter = False
min_flux_threshold = 20
num_PCs = 3
for object_id in kilonova_ids:
    if use_filter:
        meta_index = data_ob.df_metadata[data_ob.object_id_col_name] == object_id
        if not data_ob.df_metadata['filter_result'][meta_index]:
            current_coeff = np.ones((num_PCs*len(bands)))*np.inf
            current_coeff= np.insert(current_coeff,0,int(object_id))
            coeff_arr.append(current_coeff)
            print(current_coeff[0])
            continue
    event_df = data_ob.get_data_of_event(object_id)
    mid_point_dict = get_mid_pt(event_df, data_ob.band_map.keys(), separate_band_mid=separate_band_mid, min_flux_threshold=min_flux_threshold)
    values = list(mid_point_dict.values())
    lc = LightCurve(event_df, time_col_name=data_ob.time_col_name, brightness_col_name=data_ob.flux_col_name, brightness_err_col_name=data_ob.flux_err_col_name,band_col_name=data_ob.band_col_name, band_map=data_ob.band_map)
    priority_regions = lc.find_region_priority()
    priority_region1 = priority_regions[0]
    mid_point_date = np.median(np.asarray(priority_region1))
    #print(mid_point_date)
    
    #print(current_date)
    coeff_dict = predict_lc_coeff(event_df, PC_dict,separate_band_mid=separate_band_mid, min_flux_threshold= min_flux_threshold)
    #print(coeff_dict)
    current_coeff = [object_id]
    for band in bands: 
        current_coeff.extend(coeff_dict[band])
    coeff_arr.append(current_coeff)
        
    coeff_dict_list.append(coeff_dict)
    
    fig = plot_predicted_bands(coeff_dict, PC_dict, mark_maximum=mark_maximum, x_lims =[mid_point_date-80, mid_point_date+80], separate_band_mid= separate_band_mid, min_flux_threshold=min_flux_threshold)
    
    #coeff_dict = predict_alert_light_curves(current_date,event_df, PC_dict)
    #print(data_ob.get_object_type(object_id))
    #plt.savefig("kilonova_curves/fit_separated_band_lc_with_PCs/target_"+str(data_ob.get_object_type_number(object_id))+"/object_"+str(object_id)+"mid_pt_as_median_PCs_decoupled")
    plt.show()
    plt.close('all')

In [ ]:
len(coeff_arr)

In [ ]:
print(coeff_dict_list)

In [ ]:
np.save("coeff_arrays/ZTF_coeff", coeff_dict_list)

# Random current date

In [ ]:
PC_dict = get_PCs(3,all_bands=False)
start = time.time()
coeff_arr = []
event_type = []
#object_ids = data_ob.get_ids_of_event_type(6)
#fig = plt.figure()
#ax = fig.gca(projection='3d')
coeff_dict_list = []
object_list= []
separate_band_mid = True
mark_maximum = True
use_filter = False
min_flux_threshold = 20
for object_id in kilonova_ids:
    if use_filter:
        meta_index = data_ob.df_metadata[data_ob.object_id_col_name] == object_id
        if not data_ob.df_metadata['filter_result'][meta_index]:
            coeff_arr 
            continue
    
    event_df = data_ob.get_data_of_event(object_id)
    mid_point_dict = get_mid_pt(event_df, bands = bands, separate_band_mid=separate_band_mid, min_flux_threshold=min_flux_threshold)
    values = list(mid_point_dict.values())
    lc = LightCurve(data_ob, object_id)
    priority_regions = lc.find_region_priority()
    priority_region1 = priority_regions[0]
    mid_point_date = np.median(np.asarray(priority_region1))
    current_date = mid_point_date+ random()*50-25

    coeff_dict = predict_lc_coeff(event_df, PC_dict, current_date=current_date, separate_band_mid=separate_band_mid, min_flux_threshold=min_flux_threshold, bands = bands)
    coeff_dict_list.append(coeff_dict)
    object_list.append(object_id)
    fig = plot_predicted_bands(coeff_dict, PC_dict, current_date=current_date, mark_maximum=True, x_lims =[mid_point_date-100, mid_point_date+100], separate_band_mid=separate_band_mid, min_flux_threshold=min_flux_threshold, bands = bands)
    #ax=plt.gca()
    #plt.text(.01,.94,data_ob.object_id_col_name+": "+str(object_id),fontsize=15, transform=ax.transAxes)
    #if dataset_val == 0:
        #print(data_ob.get_object_type_for_PLAsTiCC(object_id))
    #    plt.text(.01,.88,"Type: "+data_ob.get_object_type_for_PLAsTiCC(object_id),fontsize=15, transform=ax.transAxes)
    #plt.legend(loc="upper right")
    
    #coeff_dict = predict_alert_light_curves(current_date,event_df, PC_dict)
    #print(data_ob.get_object_type(object_id))
    #plt.savefig("kilonova_curves/fit_separated_band_lc_with_PCs/target_"+str(data_ob.get_object_type_number(object_id))+"/object_"+str(object_id)+"mid_pt_with_random_current")
    #plt.savefig("kilonova_curves/fit_lc_with_PCs/target_"+str(data_ob.get_object_type_number(object_id))+"/object_"+str(object_id)+"highest_priority_random_curr_date")
    plt.show()
    plt.close('all')

#end = time.time()
#print(end - start)

In [ ]:
np.save("coeff_arrays/ZTF_coeff",coeff_dict_list)